<a href="https://colab.research.google.com/github/pbenight/grid-solver/blob/main/Maze_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import time
import random
from PIL import Image, ImageDraw
import json
from google.colab import files

In [ ]:
def get_barrier_locations():
  user_quits = ""
  print(   """Please input the locations where the blocks are located one at a time in X,Y format with \n
              the top left of the board being (0,0) and the bottom right being (9,9). When you are done,\n
              enter in '.' and I will begin running. Expect approximately 10 minutes for me to finish.\n
              Ex. 1,3""")
  current_barrier_locations = []
  even_count = 0
  odd_count = 0
  i = 1
  while user_quits != ".":
    user_selection = input("Please enter block number {} in x,y format".format(i))
    i += 1
    try:
      if " " in user_selection:
        choices = user_selection.split(" ")
        for choice in choices:
          x, y = choice.split(",")
          x = int(x)
          y = int(y)
          current_barrier_locations.append((x, y))
          if (x + y) % 2 == 0:
            even_count += 1
          else:
            odd_count += 1
      else:
        x, y = user_selection.split(",")
        x = int(x)
        y = int(y)
        current_barrier_locations.append((x, y))
        if (x + y) % 2 == 0:
          even_count += 1
        else:
          odd_count += 1
    except Exception:
      print("That was not an accepted input.")
    print("The current blocks are: {}".format(current_barrier_locations))
    mistake = input("If this is correct, hit 'y', if this is wrong, type anything else.")
    if mistake != "y":
      i = 1
      current_barrier_locations = []

    user_quits = input("If that's all, please enter '.'. If not, enter anything else.")
  if odd_count != even_count:
    print("This board is likely unsolveable. Continue at your own risk.")
  return current_barrier_locations

In [ ]:
def generate_random_barriers(grid_size=10, start=(0,0), num_barriers=4):
  start_x, start_y = start

  even_count = 0
  odd_count = 0

  i = 1
  barrier_locs = []

  while i <= num_barriers:
    x = random.randint(0, grid_size - 1)
    y = random.randint(0, grid_size - 1)

    if x == start_x and y == start_y: #if the barrier is at the start, skip
      continue
    elif (x,y) in barrier_locs:
      continue
    elif (x + y) % 2 == 0: #if the barrier is at an even location
      if even_count == num_barriers / 2:
        continue
      else:
        barrier_locs.append((x, y))
        even_count += 1
        i += 1
    else: # the barrier is at an odd location
      if odd_count == num_barriers / 2:
        continue
      else:
        barrier_locs.append((x, y))
        odd_count += 1
        i += 1

  barrier_locs.sort()

  return barrier_locs

In [ ]:
def generate_maze(barrier_locs=[], grid_size=10):
  """ 
  Since we're building down, I used y axis first and then filled in x axis.
  With that, when you print it, it'll be Row by row instead of Columns.
  """
  y_axis = []
  y = 0
  while y < grid_size:

    x_axis = []
    x = 0
    while x < grid_size:
      if (x,y) in barrier_locs:
        x_axis.append(999) #these are barriers
      else:
        x_axis.append(0) #these are unvisited spaces
      x += 1

    y_axis.append(x_axis)
    y += 1

  maze = y_axis
  #the highest score *should* be the total number of spaces in the maze minus the number of barriers
  max_score = (grid_size * grid_size) - len(barrier_locs) 
  return maze, max_score

In [ ]:
def get_open_positions(maze):
  """
  Returns all the positions in the maze that are zero
  currently not using the closed_positions
  """
  open_positions = []
  closed_positions = []
  for x, row in enumerate(maze):
    for y, value in enumerate(row):
      if value > 0: #since we're assuming 0 as empty, anything bigger has been visited, and therefore is closed
        closed_positions.append((x, y))
      else: #all else are open
        open_positions.append((x, y))

  return open_positions

In [ ]:
def move_south(loc):
  """
  Simple function to move the current square the direction stated in the name
  """

  x, y = loc
  return (x, y + 1)

In [ ]:
def move_north(loc):
  """
  Simple function to move the current square the direction stated in the name
  """
  x, y = loc
  return (x, y - 1)

In [ ]:
def move_east(loc):
  """
  Simple function to move the current square the direction stated in the name
  """
  x, y = loc
  return (x + 1, y)

In [ ]:
def move_west(loc):
  """
  Simple function to move the current square the direction stated in the name
  """
  x, y = loc
  return (x - 1, y)

In [ ]:
def current_neighbors(loc, open_positions):
  """
  Neighbors are a big thing, and this function assumes that we want it to be a viable/traversable neighbor. 
  This function returns all the cardinal direction neighbors of a location provided.
  It checks to make sure that the new location is part of the open_positions, so a new open_positions has to be passed in every time.
  """
  neighbors = []
  north = move_north(loc)
  south = move_south(loc)
  east = move_east(loc)
  west = move_west(loc)
  if east in open_positions:
    neighbors.append(east)
  if south in open_positions:
    neighbors.append(south)
  if west in open_positions:
    neighbors.append(west)
  if north in open_positions:
    neighbors.append(north)
  return neighbors

In [ ]:
def generate_path(maze, end):
  i, j = end
  k = maze[i][j]
  the_path = [(i,j)]
  while k > 1:
    if i > 0 and maze[i - 1][j] == k-1:
      i, j = i-1, j
      the_path.append((i, j))
      k-=1
    elif j > 0 and maze[i][j - 1] == k-1:
      i, j = i, j-1
      the_path.append((i, j))
      k-=1
    elif i < len(maze) - 1 and maze[i + 1][j] == k-1:
      i, j = i+1, j
      the_path.append((i, j))
      k-=1
    elif j < len(maze[i]) - 1 and maze[i][j + 1] == k-1:
      i, j = i, j+1
      the_path.append((i, j))
      k -= 1
  return the_path

In [ ]:
def draw_matrix(a, end, the_path = []):
  zoom = 20
  borders = 6
  start = 0,0
  images = []
  im = Image.new('RGB', (zoom * len(a[0]), zoom * len(a)), (255, 255, 255))
  draw = ImageDraw.Draw(im)
  for i in range(len(a)):
      for j in range(len(a[i])):
          color = (255, 255, 255)
          r = 0
          if a[i][j] == 999:
              color = (0, 0, 0)
          if i == start[0] and j == start[1]:
              color = (0, 255, 0)
              r = borders
          if i == end[0] and j == end[1]:
              color = (0, 255, 0)
              r = borders
          draw.rectangle((j*zoom+r, i*zoom+r, j*zoom+zoom-r-1, i*zoom+zoom-r-1), fill=color)

  for u in range(len(the_path)-1):
      y = the_path[u][0]*zoom + int(zoom/2)
      x = the_path[u][1]*zoom + int(zoom/2)
      y1 = the_path[u+1][0]*zoom + int(zoom/2)
      x1 = the_path[u+1][1]*zoom + int(zoom/2)
      draw.line((x,y,x1,y1), fill=(255, 0,0), width=5)
  draw.rectangle((0, 0, zoom * len(a[0]), zoom * len(a)), outline=(0,255,0), width=2)
  images.append(im)
  images[0].save('maze.gif',
               save_all=True, append_images=images[1:],
               optimize=False, duration=1, loop=0)

In [ ]:
def export_maze(maze, barrier_locs):
  """
  This function saves the completed mazes with the sorted barrier locations as the key
  By memoizing the end states, will save on computation for future copies of the same puzzle
  """
  maze_data = {}
  barrier_locs.sort() #make sure the keys are in ascending order
  try:
    with open("maze_data.json", 'r') as f:
      maze_data = json.load(f)
  except Exception as e:
    pass #no file exists, continue on
  barrier_locs_str = str(barrier_locs)
  maze_data[barrier_locs_str] = maze
  with open('maze_data.json', 'w') as f:
    json.dump(maze_data, f)
  return None

In [ ]:
def check_prev_solved(barrier_locs):
  try:
    with open("maze_data.json", 'r') as f:
      maze_data = json.load(f)
      barrier_locs_str = str(barrier_locs)
      if barrier_locs_str in maze_data:
        return maze_data[barrier_locs_str]
      else:
        return None
  except Exception as e:
    return None

In [ ]:
def print_maze(maze):
  local_maze = copy.deepcopy(maze)
  for y_count, column in enumerate(local_maze):
    for x_count, value in enumerate(column):
      if value == 999:
        local_maze[y_count][x_count] = "XX"
      elif value == 0:
        local_maze[y_count][x_count] = "00"
      elif value < 10:
        local_maze[y_count][x_count] = "0" + str(value)

  print('\n'.join([' | '.join([str(cell) for cell in row]) for row in local_maze]))

In [ ]:
def find_path(maze, score, max_score, loc, barrier_locs=[]):
  """

  """
  local_maze = copy.deepcopy(maze)
  x, y = loc

  score += 1
  local_maze[x][y] = score
  if score >= max_score - 1: #if we're just about done, just return the maze
    return local_maze

  open_positions = get_open_positions(local_maze) #find all open positions on the map
  next_neighbors = current_neighbors(loc, open_positions) #check the open positions available to move to

  ### Exit Cases ###

  # Need to exit this branch if there's an orphaned block, or if more than one block has only one neighbor

  single_neighbor = 0 #count for all spaces with single valid neighbors on the board

  for position in open_positions: #check every location on the board for invalid / unsolvable set-ups
    next_moves = current_neighbors(position, open_positions)
    num_poss_moves = len(next_moves)

    if num_poss_moves == 0: #if an open spot has no viable neighbors, it's an orphan
      #print("Orphan block found. Score was:{}".format(score))
      return None #break this recursive loop

    elif num_poss_moves == 1: #If an open spot has one viable neighbor, it could be the end spot
      
      # if the open spot is directly next to the current position on the board, 
      # it's not really single so don't increment (currently only set for when starting a board or completing one)
      if position in next_neighbors and (score < 2 or score > 92): 
        pass #do nothing, it's fine
      else: #if the open spot is not next to current position, it's a valid single spot
        single_neighbor += 1
      
      # Before checking any more spaces, check how many blocks have a single neighbor
      # If there are more than one, then the current board state is likely unsolvable. 
      # (should test this with more than 2 to see if that fixes some of the errors that occur from moving the piece first)
      if single_neighbor > 2:
        if score > 92: #if the score is almost at the end, feel free to exit
          print("More than two Blocks with single neighbors found. Score was:{}".format(score))
          print_maze(local_maze)
          #draw_matrix(local_maze, loc, generate_path(local_maze, loc))
          export_maze(local_maze, barrier_locs)
          print("Map has been printed")        
          raise Exception
        #elif score < 3:
        #  pass
        else:
          return None
      else:
        pass
    else:
      pass

  #print(next_neighbors)
  if next_neighbors == []:
    print("No neighbors found. Score was:{}".format(score))
    return None
  else:
    ### BEGIN RECURSION ###
    while len(next_neighbors) > 0:

      #if the program has three viable paths, the best choice is to 
      #pick one at random to prevent potentially creating issues for itself
      if len(next_neighbors) == 3: 
        random.shuffle(next_neighbors)
      else:
        pass 
      
      next_loc = next_neighbors.pop() #get the next location the piece will move to
      winner = find_path(local_maze, score, max_score, next_loc, barrier_locs) #recursively call yourself
      if winner is not None:
        print_maze(winner)
        #draw_matrix(winner, loc, generate_path(winner, next_loc))  
        export_maze(winner, barrier_locs)     
        print("Map has been printed")  
        raise Exception
      else:
        #print("Branch end: Score: {}".format(score))
        pass

In [ ]:
### MAIN PART 1 - Get Barrier Locations and Generate Maze ###

barrier_locs = get_barrier_locations()
#barrier_locs = generate_random_barriers()
barrier_locs.sort()

maze, max_score = generate_maze(barrier_locs, 12)
print(barrier_locs)
print_maze(maze)
print("\nThe max score for this maze is: {}\n".format(max_score))

Please input the locations where the blocks are located one at a time in X,Y format with 

              the top left of the board being (0,0) and the bottom right being (9,9). When you are done,

              enter in '.' and I will begin running. Expect approximately 10 minutes for me to finish.

              Ex. 1,3


In [ ]:
### MAIN PART 2 - Check if maze has been solved before, and if not, solve the maze ###

prev_solved = check_prev_solved(barrier_locs)
if prev_solved is not None:
  print("I know this one! Here's my solution. Good Luck!\n\n")
  print_maze(prev_solved)

else:
  best_score = 0
  best_maze = []
  try:
    solution = find_path(maze, 0, max_score, (0,0), barrier_locs)
  except Exception as e:
    print("I exited the loop. Solution was found! Otherwise, another issue occured {}".format(e))

KeyboardInterrupt: ignored

In [ ]:
"""
def save_to_drive():
  uploaded = drive.CreateFile({'Maze Solutions': 'maze_data.json'})
  uploaded.SetContentString('Sample upload file content')
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))
"""

"\ndef save_to_drive():\n  uploaded = drive.CreateFile({'Maze Solutions': 'maze_data.json'})\n  uploaded.SetContentString('Sample upload file content')\n  uploaded.Upload()\n  print('Uploaded file with ID {}'.format(uploaded.get('id')))\n"

In [ ]:
def fill_memory(num_rounds=10):

  i = 0

  while i < num_rounds:
    barrier_locs = generate_random_barriers()
    maze, max_score = generate_maze(barrier_locs)
    print_maze(maze)
    print(max_score)
    prev_solved = check_prev_solved(barrier_locs)

    if prev_solved is not None:
      print("I know this one! Here's my solution. Good Luck!\n\n")
      print_maze(prev_solved)

    else:
      best_score = 0
      best_maze = []
      try:
        solution = find_path(maze, 0, max_score, (0,0), barrier_locs)
      except Exception as e:
        print("I exited the loop. Solution was found!")
    i += 1

  files.download('maze_data.json')

In [ ]:
fill_memory()

00 | 00 | 00 | 00 | 00 | XX | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | XX
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | XX | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00 | 00
00 | 00 | 00 | 00 | XX | 00 | 00 | 00 | 00 | 00
96
